In [ ]:
# Installing some packages
! pip install --upgrade jax jaxlib
! pip install pmdarima
! pip install statsmodels
! pip install tensorflow
! pip install keras
! pip install scikit-optimize
! pip install scikeras

In [ ]:
# mount google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

# Loading in the packages

In [ ]:
# Standard data wrangling and plotting packages
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

# Cleaning outputs
import warnings

# Stationarity analysis packages
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import ADFTest, ndiffs

# Performance metrics packages
from sklearn.metrics import mean_squared_error
from math import sqrt

# ARIMA and cross-validation packages
from statsmodels.tsa.arima.model import ARIMA
from traitlets.traitlets import validate
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from sklearn.model_selection import TimeSeriesSplit
import itertools
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from pmdarima.arima import auto_arima

# LSTM packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten

from scikeras.wrappers import KerasRegressor
from tensorflow.keras import Model, Input
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from skopt import BayesSearchCV
import os
import logging
from tensorflow.keras import backend as K
import tensorflow as tf

# VAR/VECM packages
import statsmodels.api as sm
import statsmodels.tsa.vector_ar.vecm as vecm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.var_model import VAR as var2

# Loading and cleaning in the data

### 0. Filter Datetime Range

In [ ]:
def filtertime(dataset, start, end):
    dataset = dataset.copy()
    dataset = dataset.loc[(dataset.index >= start) & (dataset.index <= end)]
    return dataset

### 1. SPY Dataset (Kaggle)

In [ ]:
base_path = '/content/drive/My Drive/DSP/'

SPY_1m = pd.read_csv(base_path + "1_min_SPY_2008-2021.csv/1_min_SPY_2008-2021.csv", index_col=False, parse_dates=['date'], infer_datetime_format=True)
SPY_1m = SPY_1m[["date", "close"]]
SPY_1m.drop_duplicates(inplace=True)
SPY_1m = SPY_1m.sort_values(by='date')
SPY_1m.reset_index(drop=True, inplace=True)
SPY_1m.index = SPY_1m.pop('date')
SPY_1m = filtertime(SPY_1m, '2010-01-01 07:30:00', '2014-01-01 07:30:00')
print(SPY_1m.isna().sum().sum())
print(SPY_1m)
SPY_1m.plot()
plt.title('SPY Stock Price Time Series: 1min Frequency')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.show()

### 2. Synthetic random walk time series

In [ ]:
# Seed for reproducibility
np.random.seed(42)

# Generate datetime index for one year with one-minute intervals
start_date = pd.Timestamp("2023-01-01 00:00:00")
end_date = pd.Timestamp("2023-12-31 23:59:00")
date_range = pd.date_range(start=start_date, end=end_date, freq='T')

# Generate random walk data spanning the length of date_range
initial_value = 0
n_points_new = len(date_range)
steps_new = np.random.choice([-1, 1], size=n_points_new-1)  # Subtract 1 for the initial value
time_series_new = np.concatenate(([initial_value], np.cumsum(steps_new)))

# Create DataFrame with the datetime index
rw_df = pd.DataFrame(time_series_new, columns=['Value'], index=date_range)
rw_df.rename(columns={"Value":"close"}, inplace=True)
print(rw_df.isna().sum().sum())
print(rw_df)
# Plot the generated random walk data
# plt.figure(figsize=(12, 6))
rw_df.plot()
plt.title('Random Walk Time Series: 1min Frequency')
plt.xlabel('Date')
plt.ylabel('Close')
plt.grid(True)
plt.show()

### 3. Reyes financial dataset (Euro-dollar Exchange)

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.styles.stylesheet")

base_path = '/content/drive/My Drive/DSP/'

Reyes_2005 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2005.xlsx", index_col=False, header=None))[[0,4]]
Reyes_2006 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2006.xlsx", index_col=False, header=None))[[0,4]]
Reyes_2007 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2007.xlsx", index_col=False, header=None))[[0,4]]
Reyes_2008 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2008.xlsx", index_col=False, header=None))[[0,4]]
Reyes_2009 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2009.xlsx", index_col=False, header=None))[[0,4]]
Reyes_2010 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2010.xlsx", index_col=False, header=None))[[0,4]]
Reyes_2011 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2011.xlsx", index_col=False, header=None))[[0,4]]
Reyes_2012 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2012.xlsx", index_col=False, header=None))[[0,4]]
Reyes_2013 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2013.xlsx", index_col=False, header=None))[[0,4]]
Reyes_2014 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2014.xlsx", index_col=False, header=None))[[0,4]]
Reyes_2015 = pd.DataFrame(pd.read_excel(base_path + "Reyes/EURUSD_M1_2015.xlsx", index_col=False, header=None))[[0,4]]

In [ ]:
Reyes_1m = pd.concat([Reyes_2005,Reyes_2006,Reyes_2007,Reyes_2008,Reyes_2009,Reyes_2010,Reyes_2011,Reyes_2012,Reyes_2013,Reyes_2014,Reyes_2015], ignore_index=True)
Reyes_1m.rename(columns={0:"date", 4:"close"}, inplace=True)
Reyes_1m.drop_duplicates(inplace=True)
Reyes_1m = Reyes_1m.sort_values(by="date")
Reyes_1m.reset_index(drop=True, inplace=True)
Reyes_1m.index = Reyes_1m.pop("date")
Reyes_1m = filtertime(Reyes_1m, '2006-01-01 00:00:00', '2014-12-31 23:59:00')
Reyes_5m = Reyes_1m.resample("5Min").nearest()
print(Reyes_1m.isna().sum().sum())
print(Reyes_5m.isna().sum().sum())
print(Reyes_5m)
Reyes_5m.plot()
plt.title('EUR-USD Forex Exchange Rate Time Series: 5min Frequency')
plt.xlabel('Date')
plt.ylabel('Close Rate')
plt.grid(True)
plt.show()

### 4. Bitcoin Dataset (Kaggle)

In [ ]:
base_path = '/content/drive/My Drive/DSP/'

BTC_2017 = pd.DataFrame(pd.read_csv(base_path + 'BTC/BTC-2017min.csv', index_col=False, header=None, low_memory=False))[[1,6]]
BTC_2018 = pd.DataFrame(pd.read_csv(base_path + 'BTC/BTC-2018min.csv', index_col=False, header=None, low_memory=False))[[1,6]]
BTC_2019 = pd.DataFrame(pd.read_csv(base_path + 'BTC/BTC-2019min.csv', index_col=False, header=None, low_memory=False))[[1,6]]
BTC_2020 = pd.DataFrame(pd.read_csv(base_path + 'BTC/BTC-2020min.csv', index_col=False, header=None, low_memory=False))[[1,6]]
BTC_2021 = pd.DataFrame(pd.read_csv(base_path + 'BTC/BTC-2021min.csv', index_col=False, header=None, low_memory=False))[[1,6]]

In [ ]:
BTC_1m = pd.concat([BTC_2017, BTC_2018, BTC_2019, BTC_2020, BTC_2021], ignore_index=True)
BTC_1m.rename(columns={1:"date", 6:"close"}, inplace=True)
BTC_1m['close'] = pd.to_numeric(BTC_1m['close'], errors='coerce')
BTC_1m.drop_duplicates(inplace=True)
BTC_1m = BTC_1m.sort_values(by="date")
BTC_1m.reset_index(drop=True, inplace=True)
BTC_1m = BTC_1m.iloc[:-1]
BTC_1m['date'] = pd.to_datetime(BTC_1m['date'])
BTC_1m.index = BTC_1m.pop("date")
BTC_1m = filtertime(BTC_1m, '2017-01-01 00:01:00', '2021-12-31 23:59:00')
print(BTC_1m.isna().sum().sum())
print(BTC_1m)
BTC_1m.plot()
plt.title('BTC Cryptocurrency Price Time Series: 1min Frequency')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.show()

### Replace zeros with small value

In [ ]:
def replace(df):
  df = df.copy()
  # print((df == 0).sum().sum())
  locations = df.where(df==0).stack().index.tolist()
  # print(locations)
  df = df.replace(0, 0.0001)
  # print((df == 0).sum().sum())
  return df

# Raw to symbolic boxplot time series

In [ ]:
def rawtosymbolic(dataset, f, price):
    dataset = dataset.copy()
    # print(dataset.isna().sum().sum())
    # print(dataset.dtypes)

    # symbolic descriptive statistics
    min_data = (dataset.groupby(pd.Grouper(freq=f, origin="start_day")).min().dropna()).rename(columns={'close':'min'})
    q25_data = dataset.groupby(pd.Grouper(freq=f, origin="start_day")).quantile(q=0.25).dropna()
    q50_data = dataset.groupby(pd.Grouper(freq=f, origin="start_day")).quantile().dropna()
    q75_data = dataset.groupby(pd.Grouper(freq=f, origin="start_day")).quantile(q=0.75).dropna()
    # min_data = (q25_data - (1.5*(q75_data - q25_data))).rename(columns={'close':'min'})
    # max_data = (q75_data + (1.5*(q75_data - q25_data))).rename(columns={'close':'max'})
    max_data = (dataset.groupby(pd.Grouper(freq=f, origin="start_day")).max().dropna()).rename(columns={'close':'max'})

    # max_data -= q75_data
    q75_data -= q50_data
    q50_data -= q25_data
    # q25_data -= min_data

    # print(q50_data.isna().sum().sum())
    # print(q25_data.isna().sum().sum())
    # print(q75_data.isna().sum().sum())
    # print(len(q50_data))
    # print(len(q25_data))
    # print(len(q75_data))

    # symbolic table
    sym_table = pd.DataFrame(columns=["Q1", "Q2", "Q3"])
    # sym_table["min"] = min_data[price]
    sym_table["Q1"] = q25_data[price]
    sym_table["Q2"] = q50_data[price]
    sym_table["Q3"] = q75_data[price]
    # sym_table["max"] = max_data[price]

    # filter original dataframe to price column and single point average using mean
    og_table = dataset[[price]].copy()
    sp_table = dataset.groupby(pd.Grouper(freq=f, origin="start_day")).mean().dropna()
    sp_table = sp_table[[price]]

    return sym_table, sp_table, og_table, min_data, max_data

def math_coherence(temp):
    temp = temp.copy()
    df = pd.DataFrame(temp)
    df.columns = ['Q1', 'Q2', 'Q3']
    # df['Q1'] += df['min']
    df['Q2'] += df['Q1']
    df['Q3'] += df['Q2']
    # df['max'] += df['Q3']
    return df

In [ ]:
# Synthetic data
rw_sym_data, rw_sp_data, rw_raw_data, rw_min_data, rw_max_data = rawtosymbolic(rw_df, "1D", "close")
rw_sym_data = replace(rw_sym_data)
rw_sp_data = replace(rw_sp_data)
rw_raw_data = replace(rw_raw_data)

rw_sp_data.plot(label='close')
plt.title('Random Walk, Freq:1day, Agg:mean')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.show()

ax = math_coherence(rw_sym_data).plot()
rw_min_data.plot(linestyle='--', ax=ax, linewidth=0.5, alpha=0.8)
rw_max_data.plot(linestyle='--', ax=ax, linewidth=0.5, alpha=0.8)
plt.title('Random Walk: Freq:1day, Agg: 3 boxplot variables')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.show()

In [ ]:
# S&P 500 dataset (Stock)
spy_sym_data, spy_sp_data, spy_raw_data, spy_min_data, spy_max_data = rawtosymbolic(SPY_1m, "1D", "close")
spy_sym_data = replace(spy_sym_data)
spy_sp_data = replace(spy_sp_data)
spy_raw_data = replace(spy_raw_data)

spy_sp_data.plot()
plt.title('SPY, Freq:1day, Agg:mean')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.show()

ax = filtertime(math_coherence(spy_sym_data), '2011-01-01', '2011-02-01').plot()
filtertime(spy_min_data, '2011-01-01', '2011-02-01').plot(linestyle='--', ax=ax, linewidth=0.5, alpha=0.8)
filtertime(spy_max_data, '2011-01-01', '2011-02-01').plot(linestyle='--', ax=ax, linewidth=0.5, alpha=0.8)
plt.title('SPY: Freq:1day, Agg: 3 boxplot variables')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.show()

In [ ]:
# Eurodollar exchange dataset (Reyes et al.)
reyes_sym_data, reyes_sp_data, reyes_raw_data, reyes_min_data, reyes_max_data = rawtosymbolic(Reyes_5m, "1D", "close")
reyes_sym_data = replace(reyes_sym_data)
reyes_sp_data = replace(reyes_sp_data)
reyes_raw_data = replace(reyes_raw_data)

reyes_sp_data.plot()
plt.title('EUR-USD, Freq:1day, Agg:mean')
plt.xlabel('Date')
plt.ylabel('Close Rate')
plt.grid(True)
plt.show()

ax = filtertime(math_coherence(reyes_sym_data), '2005-01-01', '2015-01-01').plot()
filtertime(reyes_min_data, '2005-01-01', '2015-01-01').plot(linestyle='--', ax=ax, linewidth=0.5, alpha=0.8)
filtertime(reyes_max_data, '2005-01-01', '2015-01-01').plot(linestyle='--', ax=ax, linewidth=0.5, alpha=0.8)
plt.title('EUR-USD: Freq:1day, Agg: 3 boxplot variables')
plt.xlabel('Date')
plt.ylabel('Close Rate')
plt.grid(True)
plt.show()

In [ ]:
# Bitcoin Dataset (Cryptocurrency)
BTC_sym_data, BTC_sp_data, BTC_raw_data, BTC_min_data, BTC_max_data = rawtosymbolic(BTC_1m, "1D", "close")
BTC_sym_data = replace(BTC_sym_data)
BTC_sp_data = replace(BTC_sp_data)
BTC_raw_data = replace(BTC_raw_data)

BTC_sp_data.plot()
plt.title('BTC, Freq:1day, Agg:mean')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.show()

ax = filtertime(math_coherence(BTC_sym_data), '2017-01-01','2022-01-01').plot()
filtertime(BTC_min_data, '2017-01-01','2022-01-01').plot(linestyle='--', ax=ax, linewidth=0.5, alpha=0.8)
filtertime(BTC_max_data, '2017-01-01','2022-01-01').plot(linestyle='--', ax=ax, linewidth=0.5, alpha=0.8)
plt.title('BTC: Freq:1day, Agg: 3 boxplot variables')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.show()

# Train-Test-Split the data

In [ ]:
# Manually defined function to prevent data leakage
def traintestsplit(df, val_size, test_size):
    df = df.copy()
    test_size_ = int(len(df)*(1-test_size))
    df_test = df[test_size_:]
    if val_size == 0:
      df_train = df[:test_size_]
      return df_train, df_test
    else:
      val_size_ = int(len(df)*(1-test_size-val_size))
      df_val = df[val_size_:test_size_]
      df_train = df[:val_size_]
      return df_train, df_val, df_test

# Stationarity analysis functions

In [ ]:
# This function checks the stationarity of the input time series
def check_stationarity(df):
    df = df.copy()
    # rolling statistics
    rolling_mean = df.rolling(window = 12).mean()
    rolling_std = df.rolling(window = 12).std()
    # rolling statistics plot
    plt.plot(df, color = 'blue', label = 'Original')
    plt.plot(rolling_mean, color = 'red', label = 'Rolling Mean')
    plt.plot(rolling_std, color = 'black', label = 'Rolling Std')
    plt.xticks(rotation=45)
    plt.legend(loc = 'best')
    plt.title('Rolling Mean & Rolling Standard Deviation')
    plt.show()
    # acf plot
    plot_acf(df)
    plt.show()
    # pacf_plot
    plot_pacf(df)
    plt.show()
    # Dickey-Fuller statistical test
    result = adfuller(df.dropna())
    # print('ADF Statistic: {}'.format(result[0]))
    print('p-value: {}'.format(result[1]))
    # print('Critical Values:')
    # for key, value in result[4].items():
        # print('\t{}: {}'.format(key, value))

    # if p-value is over threshold (and ADF statistic is far from the critical values)
    threshold = 0.05
    if result[1] > threshold:
        print('Time series is likely non-stationary.')
    else:
        print('Time series is likely stationary')

    return

In [ ]:
# This function explores different transformations of the dataset, and checks the stationarity for each transformation.

def transformations(df, order):
    df = df.copy()
    # find logarithm of time series
    df_log = np.log(df)
    print("logarithm plot")
    plt.plot(df_log)
    plt.xticks(rotation=45)
    plt.show()
    check_stationarity(df_log)
    # subtract rolling mean
    rolling_mean = df.rolling(window=12).mean()
    df_log_minus_mean = df_log - rolling_mean
    df_log_minus_mean.dropna(inplace=True)
    print("Rolling mean,std and ACF plots")
    check_stationarity(df_log_minus_mean)
    # subtract rolling mean exponential decay
    # rolling_mean_exp_decay = df.ewm(halflife=12, min_periods=0, adjust=True).mean()
    # df_log_exp_decay = df_log - rolling_mean_exp_decay
    # df_log_exp_decay.dropna(inplace=True)
    # check_stationarity(df_log_exp_decay)
    # find differenced time series
    fig1, ax1 = plt.subplots()
    ax1.set_title('{} Order Differencing'.format(order))
    diff_df = df_log.copy()
    for i in range(order):
        diff_df = diff_df.diff()
    ax1.plot(diff_df)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Differenced Value')
    plt.tight_layout()
    plt.show()
    fig2, (ax2, ax3) = plt.subplots(1, 2)
    plot_pacf(diff_df.dropna(), ax=ax2)
    ax2.set_title('PACF')
    plot_acf(diff_df.dropna(), ax=ax3)
    ax3.set_title('ACF')

    # f = plt.figure()
    # ax1 = f.add_subplot(121)
    # ax1.set_title('{} Order Differencing'.format(order))
    # diff_df = df.copy()
    # for i in range(order):
    #     diff_df = diff_df.diff()
    # ax1.plot(diff_df)
    # ax2 = f.add_subplot(122)
    # plot_pacf(diff_df.dropna(), ax=ax2)
    plt.show()
    check_stationarity(diff_df)
    return

In [ ]:
# This function can be used to confirm the differencing term, d, using black-box statistical tests.

def ADF_KPSS_PP(df):
    df = df.copy()
    # Test whether we should difference at the alpha=0.05 significance level
    adf_test = ADFTest(alpha=0.05)
    p_val, should_diff = adf_test.should_diff(df)

    # Estimate the number of differences using an ADF, KPSS, and PP test:
    n_adf = ndiffs(df, test='adf')
    n_kpss = ndiffs(df, test='kpss')
    n_pp = ndiffs(df, test='pp')

    print("p_val: ", p_val)
    print("should_diff: ", should_diff)
    print("n_adf: ", n_adf)
    print("n_kpss: ", n_kpss)
    print("n_pp: ", n_pp)

# Performance Metrics

In [ ]:
def sMAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Compute the denominator, which is the sum of the absolute values of the actual and predicted values
    denominator = np.abs(y_true) + np.abs(y_pred)
    # Exclude cases where the denominator is zero to avoid division by zero
    mask = denominator != 0
    smape_value = (100/len(y_true)) * np.sum(np.abs(y_true[mask] - y_pred[mask]) / denominator[mask])
    return round(smape_value, 4)

def MMRE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Exclude cases where y_true is zero to avoid division by zero
    mask = y_true != 0
    mmre_value = 100*np.mean(np.abs(y_true[mask] - y_pred[mask]) / y_true[mask])

    # Rounding to 3 significant figures
    return round(mmre_value, 4)

def performance(observed, forecasts):
    rmses = []
    smapes = []
    mmres = []

    for col in observed.columns:
        obs_col = observed[col]
        fcast_col = forecasts[col]

        rmse = round(sqrt(mean_squared_error(obs_col, fcast_col)),4)
        smape = sMAPE(obs_col, fcast_col)
        mmre = MMRE(obs_col, fcast_col)

        print(f"Performance for column {col}:")
        print("\tRMSE:", rmse)
        print("\tsMAPE:", smape)
        print("\tMMRE:", mmre)
        print("--------")

        rmses.append(rmse)
        smapes.append(smape)
        mmres.append(mmre)

    print("Overall Performance:")
    print("\tAverage RMSE:", round(np.mean(rmses),4))
    print("\tAverage sMAPE:", round(np.mean(smapes),4))
    print("\tAverage MMRE:", round(np.mean(mmres), 4))

In [ ]:
# forecast = pd.read_csv(base_path + '/save_forecasts/btc_arima_ms_forecasted.csv')
# data = BTC_sym_data
# train, test_df = traintestsplit(data, val_size=0, test_size=0.1)
# test = math_coherence(test_df)

# test['min'] = (test['Q1'] - (1.5*(test['Q3'] - test['Q1'])))
# test['max'] = (test['Q3'] + (1.5*(test['Q3'] - test['Q1'])))
# forecast['min'] = (forecast['Q1'] - (1.5*(forecast['Q3'] - forecast['Q1'])))
# forecast['max'] = (forecast['Q3'] + (1.5*(forecast['Q3'] - forecast['Q1'])))

# performance(test, forecast)

# Univariate Forecasting

### ARIMA

##### Visualize the data

In [ ]:
data = reyes_sym_data.copy()

In [ ]:
train, test = traintestsplit(data['q25'], val_size=0, test_size=0.1)

plt.plot(train.index, train)
plt.plot(test.index, test)
plt.legend(['Train', 'Test'])
plt.ylabel('Close Price')
plt.xlabel('Time')
plt.xticks(rotation=45)
plt.title('Data Visualisation')

##### manual statistical (stationarity analysis functions)

In [ ]:
print("--CHECK STATIONARITY--")
check_stationarity(train)

In [ ]:
print("--EXPLORE TRANSFORMATIONS--")
transformations(train, 1)

In [ ]:
print("--CONFIRM DIFFERENCING ORDER--")
ADF_KPSS_PP(train)

In [ ]:
ms_order = [(0,1,1)]

##### Model evaluation functions

In [ ]:
def quick_test(train, test, param, f):
    history = train.copy()
    history.index = history.index.to_period(f)
    arima = ARIMA(history, order=param).fit()
    # residuals = arima.resid[1:]
    # fig, ax = plt.subplots(1,2)
    # # Check that the residuals look random and general.
    # residuals.plot(title='Residuals', ax=ax[0])
    # # Check that the density looks normally distributed, with a mean of around zero.
    # residuals.plot(title='Density', kind='kde', ax=ax[1])
    # plt.show()
    # # Check that the lower lags barely show any significant spikes. This verifies that the residuals are close to white noise.
    # acf_res = plot_acf(residuals)
    # pacf_res = plot_pacf(residuals)

    pred = arima.get_prediction(start=train.index[0], end=train.index[-1], dynamic=False).predicted_mean
    pred.index = train.index
    pred = pred.iloc[1:]

    history = history.values.flatten().tolist()
    forecasts = []
    for t in range(0, len(test)-4, 5):
        arima = ARIMA(history, order=param).fit()
        forecasts.extend(arima.get_forecast(steps=5).predicted_mean)
        for i in range(5):
            history.append(test.iloc[t+i])
    forecasts_df = pd.DataFrame(forecasts, index=test.index[:len(forecasts)])
    # print(arima.summary())

    return pred, forecasts_df

In [ ]:
# from pmdarima import ARIMA
# from statsmodels.tsa.arima.model import ARIMA as smARIMA
# import warnings
# from statsmodels.tools.sm_exceptions import ConvergenceWarning
# warnings.simplefilter('ignore', ConvergenceWarning)

# def quick_test(train, test, order, f):

#     history = train.copy()
#     history.index = history.index.to_period(f)

#     arima = ARIMA(order=order).fit(history)
#     smarima = smARIMA(history, order=order).fit()

#     # residuals = pd.Series(arima.resid(), index=history.index[1:])
#     # fig, ax = plt.subplots(1,2)
#     # residuals.plot(title='Residuals', ax=ax[0])
#     # residuals.plot(title='Density', kind='kde', ax=ax[1])
#     # plt.show()

#     # plot_acf(residuals)
#     # plot_pacf(residuals)

#     pred = smarima.get_prediction(start=train.index[0], end=train.index[-1], dynamic=False).predicted_mean
#     pred.index = train.index
#     pred = pred.iloc[1:]

#     forecasts = []
#     for t in test.index:
#         forecasts.append(arima.predict(n_periods=1)[0])
#         arima.update(test.loc[[t]])

#     forecasts_df = pd.DataFrame(forecasts, index=test.index)

#     return pred, forecasts_df

In [ ]:
# pred, forecasts_df = quick_test(train, test['q25'], (3,1,0), 'D')
# plt.figure(figsize=(10, 6))
# plt.plot(train.index, train, label='Train')
# plt.plot(test.index, test, label='Test')
# plt.plot(forecasts_df.index, forecasts_df, label='Forecast')
# plt.xticks(rotation=45)

# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
def plot_forecasts(train, test, ms_pred_df, aa_pred_df, cv_pred_df, ms_forecast_df, aa_forecast_df, cv_forecast_df):

    forecast_dfs = [ms_forecast_df, cv_forecast_df, aa_forecast_df]
    forecast_labels = ['VS Testing Predictions', 'CV Testing Predictions', 'AA Testing Predictions']

    if len(test.columns) != 1:
        test['min'] = (test['Q1'] - (1.5*(test['Q3'] - test['Q1'])))
        test['max'] = (test['Q3'] + (1.5*(test['Q3'] - test['Q1'])))
        ms_forecast_df['min'] = (ms_forecast_df['Q1'] - (1.5*(ms_forecast_df['Q3'] - ms_forecast_df['Q1'])))
        ms_forecast_df['max'] = (ms_forecast_df['Q3'] + (1.5*(ms_forecast_df['Q3'] - ms_forecast_df['Q1'])))
        cv_forecast_df['min'] = (cv_forecast_df['Q1'] - (1.5*(cv_forecast_df['Q3'] - cv_forecast_df['Q1'])))
        cv_forecast_df['max'] = (cv_forecast_df['Q3'] + (1.5*(cv_forecast_df['Q3'] - cv_forecast_df['Q1'])))
        aa_forecast_df['min'] = (aa_forecast_df['Q1'] - (1.5*(aa_forecast_df['Q3'] - aa_forecast_df['Q1'])))
        aa_forecast_df['max'] = (aa_forecast_df['Q3'] + (1.5*(aa_forecast_df['Q3'] - aa_forecast_df['Q1'])))

        for forecast_df, forecast_label in zip(forecast_dfs, forecast_labels):
            plt.figure(figsize=(10, 6))
            plt.plot(test.index, test, label='Testing Observations', lw=2.0, color='orange')
            plt.plot(forecast_df.index, forecast_df, label=forecast_label, color='green')

            orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Observations')
            green_line = mlines.Line2D([], [], color='green', markersize=15, label=forecast_label)
            plt.legend(handles=[orange_line, green_line])

            plt.title(f'All symbolic rates ({forecast_label})')
            plt.grid(True)
            plt.show()

    for i,col in enumerate(test.columns):
        for forecast_df, forecast_label in zip(forecast_dfs, forecast_labels):
            plt.figure(figsize=(10, 6))
            plt.plot(test.index, test[col], label=f'{col} Testing Observations', lw=2.0, color='orange')
            plt.plot(forecast_df.index, forecast_df[col], label=f'{col} {forecast_label}', color='green')
            plt.legend()
            plt.title(f'{col} rates ({forecast_label})')
            plt.grid(True)
            plt.show()

    print("MS:")
    performance(test[:-4], ms_forecast_df)
    print("AA:")
    performance(test[:-4], aa_forecast_df)
    print("CC:")
    performance(test[:-4], cv_forecast_df)

    # ms_forecast_df.to_csv(base_path + '/save_forecasts/spy_arima_sp_ms_forecasted.csv', index=False)
    # aa_forecast_df.to_csv(base_path + '/save_forecasts/spy_arima_sp_aa_forecasted.csv', index=False)
    # cv_forecast_df.to_csv(base_path + '/save_forecasts/spy_arima_sp_cv_forecasted.csv', index=False)

##### CV

In [ ]:
# from traitlets.traitlets import validate
# import concurrent.futures
# from concurrent.futures import ProcessPoolExecutor
# import pandas as pd
# import numpy as np
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import TimeSeriesSplit
# import itertools
# import warnings

# def evaluate_arima_model(df, order, f):
#         df = df.copy()
#         error_scores = []
#         tscv = TimeSeriesSplit(n_splits=3)
#         for train_index, val_index in tscv.split(df):
#             train, val = df.iloc[train_index], df.iloc[val_index]
#             with warnings.catch_warnings():
#                 warnings.filterwarnings("ignore") # ignore all warnings
#                 pred, forecasts = quick_test(train, val, order, f)
#                 error = mean_squared_error(val, forecasts)
#                 error_scores.append(error)
#         return order, np.mean(error_scores)

# def cv_approach(df, f):
#     # Define the p, d and q parameters to take any value between 1 and 5
#     p = d = q = range(0, 3)

#     # Generate all different combinations of p, q and q triplets
#     pdq = list(itertools.product(p, d, q))

#     best_score, best_cfg = float("inf"), None
#     with concurrent.futures.ProcessPoolExecutor() as executor:
#         futures = {executor.submit(evaluate_arima_model, df, order, f): order for order in pdq}
#         for future in concurrent.futures.as_completed(futures):
#             order, mse = future.result()
#             if mse < best_score:
#                 best_score, best_cfg = mse, order
#             # print('ARIMA%s MSE=%.3f' % (order, mse))

#     # print('Best ARIMA=%s MSE=%.3f' % (best_cfg, best_score))
#     return best_cfg

In [ ]:
# out = cv_approach(train, f='D')
# out

In [ ]:
warnings.simplefilter('ignore', ConvergenceWarning)

def evaluate_arima_model(df, order, f):
    df = df.copy()
    error_scores = []
    tscv = TimeSeriesSplit(n_splits=3)
    for train_index, val_index in tscv.split(df):
        train, val = df.iloc[train_index], df.iloc[val_index]
        print('progress..')
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")  # ignore all warnings
            pred, forecasts = quick_test(train, val, order, f)
            error = MMRE(val, forecasts)
            error_scores.append(error)
    return order, np.mean(error_scores)

def stepwise_search(df, f):
    p = d = q = [0, 1, 1]
    best_score = float("inf")
    best_cfg = (0, 0, 0)  # initial configuration

    improved = True  # flag to check if any improvements in the loop
    while improved:
        improved = False
        neighbors = get_neighbors(best_cfg, p, d, q)

        # Parallelize evaluations of neighbors
        with concurrent.futures.ProcessPoolExecutor() as executor:
            futures = {executor.submit(evaluate_arima_model, df, order, f): order for order in neighbors}
            for future in concurrent.futures.as_completed(futures):
                order, mse = future.result()
                if mse < best_score:
                    improved = True
                    best_score = mse
                    best_cfg = order

    return best_cfg

def get_neighbors(order, p_values, d_values, q_values):
    neighbors = []

    p, d, q = order
    if p+1 in p_values:
        neighbors.append((p+1, d, q))
    if p-1 in p_values:
        neighbors.append((p-1, d, q))
    if d+1 in d_values:
        neighbors.append((p, d+1, q))
    if d-1 in d_values:
        neighbors.append((p, d-1, q))
    if q+1 in q_values:
        neighbors.append((p, d, q+1))
    if q-1 in q_values:
        neighbors.append((p, d, q-1))

    return neighbors

In [ ]:
# new = stepwise_search(train,'D')
# new

##### Auto Arima

In [ ]:
# from pmdarima.arima import auto_arima

# def AutoArima(train, test, f):
#     history = train.copy()
#     history.index = history.index.to_period(f)
#     forecast_auto = []
#     best_params = []
#     model3 = auto_arima(history, stepwise=False, seasonal=False, n_jobs=-1)
#     pred_auto = model3.predict_in_sample()
#     pred_auto.index = train.index
#     pred_auto = pred_auto.iloc[1:]
#     history = history.values.flatten().tolist()
#     for t in range(len(test)):
#         model3 = auto_arima(history, stepwise=False, seasonal=False, n_jobs=-1)
#         forecast_auto.append(model3.predict(n_periods=1)[0])
#         history.append(test.iloc[t,0])
#         model3.summary()
#         best_params.append(model3.order)
#     forecast_auto_df = pd.DataFrame(forecast_auto, index=test.index)
#     return best_params, pred_auto, forecast_auto_df

In [ ]:
warnings.simplefilter('ignore', ConvergenceWarning)

def AutoArima(train, test, f):
    history = train.copy()
    history.index = history.index.to_period(f)
    forecast_auto = []
    model3 = auto_arima(history, stepwise=True, seasonal=False, n_jobs=1)
    pred_auto = model3.predict_in_sample()
    pred_auto.index = train.index
    pred_auto = pred_auto.iloc[1:]
    best_params = model3.order
    # for t in test.index:
    #     forecast_auto.append(model3.predict(n_periods=1)[0])
    #     model3.update(test.loc[[t]])
    # forecast_auto_df = pd.DataFrame(forecast_auto, index=test.index)
    # return best_params, pred_auto, forecast_auto_df
    for t in range(0, len(test)-4, 5):
        forecast_auto.extend(model3.predict(n_periods=5))
        model3.update(test.iloc[t:t+5])
    forecast_auto_df = pd.DataFrame(forecast_auto, index=test.index[:len(forecast_auto)])
    return best_params, pred_auto, forecast_auto_df

In [ ]:
# best_params, pred_auto, forecast_auto = AutoArima(train, test, f='D')
# best_params

In [ ]:
# plt.figure(figsize=(10, 6))
# plt.plot(train.index, train, label='Train')
# plt.plot(test.index, test, label='Test')
# plt.plot(forecast_auto.index, forecast_auto, label='Forecast')
# plt.xticks(rotation=45)

# plt.legend()
# plt.grid(True)
# plt.show()

##### Arima pipeline

In [ ]:
def arima_pipeline(data, ms_order, cv_order, f):
    train, test = traintestsplit(data, val_size=0, test_size=0.1)
    ms_pred_, cv_pred_, aa_pred_, ms_forecast_, cv_forecast_, aa_forecast_ = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    stationary_train, stationary_test = train.copy(), test.copy()
    stationary_train, stationary_test = np.log(stationary_train), np.log(stationary_test)
    for i, col in enumerate(train.columns):
        aa_order, aa_pred, aa_forecast = AutoArima(train[col], test[col], f)
        print('aa params {}: {}'.format(col, aa_order))
        aa_forecast_ = pd.concat([aa_forecast_,aa_forecast], axis=1)
        aa_pred_ = pd.concat([aa_pred_,aa_pred], axis=1)

        ms_pred, ms_forecast = quick_test(stationary_train[col], stationary_test[col], ms_order[i], f)
        ms_pred_ = pd.concat([ms_pred_,ms_pred], axis=1)
        ms_forecast_ = pd.concat([ms_forecast_,ms_forecast], axis=1)

        # cv_order = stepwise_search(train[col], f)
        print('cv params {}: {}'.format(col, cv_order[i]))
        cv_pred, cv_forecast = quick_test(train[col], test[col], cv_order[i], f)
        cv_pred_ = pd.concat([cv_pred_,cv_pred], axis=1)
        cv_forecast_ = pd.concat([cv_forecast_,cv_forecast], axis=1)

    aa_pred_.columns, aa_forecast_.columns = train.columns, test.columns
    ms_pred_.columns, ms_forecast_.columns = train.columns, test.columns
    cv_pred_.columns, cv_forecast_.columns = train.columns, test.columns

    ms_pred_, ms_forecast_ = np.exp(ms_pred_), np.exp(ms_forecast_)

    if len(train.columns) == 1:
        plot_forecasts(train, test, ms_pred_, aa_pred_, cv_pred_, ms_forecast_, aa_forecast_, cv_forecast_)
    else:
        ms_pred_df = math_coherence(ms_pred_)
        aa_pred_df = math_coherence(aa_pred_)
        ms_forecast_df = math_coherence(ms_forecast_)
        aa_forecast_df = math_coherence(aa_forecast_)
        train_df = math_coherence(train)
        test_df = math_coherence(test)
        cv_pred_df = math_coherence(cv_pred_)
        cv_forecast_df = math_coherence(cv_forecast_)
        plot_forecasts(train_df, test_df, ms_pred_df, aa_pred_df, cv_pred_df, ms_forecast_df, aa_forecast_df, cv_forecast_df)

In [ ]:
# ms_order = [(1,1,1)]
# cv_order = [(1,0,0)]
ms_order = [(2,0,0),(1,0,1),(1,0,1)]
cv_order = [(0,0,1),(0,0,0),(0,0,0)]

In [ ]:
arima_pipeline(reyes_sym_data.copy(), ms_order, cv_order, 'D')

## LSTM

##### LSTM functions

In [ ]:
def df_to_windowed_df(df, window_size):
    df = df.copy()
    windowed_data = []
    target_data = []
    date_data = df.index
    for i in range(window_size, len(df)):
        window = df.iloc[i-window_size:i]
        target = df.iloc[i].values
        windowed_data.append(window)
        target_data.append(target)

    date_data, windowed_data, target_data = np.array(date_data), np.array(windowed_data), np.array(target_data)

    return date_data[window_size:], windowed_data, target_data

def plot_LSTM(dates_test, test_predictions, y_test, col):
    # plt.plot(dates_train, train_predictions, color='magenta', label = 'Training Predictions')
    # plt.plot(dates_train, y_train, color='red', label='Training Observations')
    # plt.xticks(rotation=45)
    # red_line = mlines.Line2D([], [], color='red', markersize=15, label='Training Observations')
    # magenta_line = mlines.Line2D([], [], color='magenta', markersize=15, label='Training Predictions')
    # plt.legend(handles=[red_line, magenta_line])
    # plt.title(f'{col} price')
    # plt.show()

    # plt.plot(dates_val, val_predictions, color='peachpuff', label='Validation Predictions')
    # plt.plot(dates_val, y_val, color='lightblue', label='Validation Observations')
    # plt.xticks(rotation=45)
    # lightblue_line = mlines.Line2D([], [], color='lightblue', markersize=15, label='Validation Observations')
    # peachpuff_line = mlines.Line2D([], [], color='peachpuff', markersize=15, label='Validation Predictions')
    # plt.legend(handles=[lightblue_line, peachpuff_line])
    # plt.title(f'{col} price')
    # plt.show()

    plt.plot(dates_test, y_test, color='orange', label='Testing Observations')
    plt.plot(dates_test, test_predictions, color='green', label='Testing Predictions')
    plt.xticks(rotation=45)
    orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Observations')
    green_line = mlines.Line2D([], [], color='green', markersize=15, label='Testing Predictions')
    plt.legend(handles=[orange_line, green_line])
    plt.title(f'{col} rates')
    # plt.grid(True)
    plt.show()

    # plt.plot(dates_train, train_predictions, color='magenta', label = 'Training Predictions')
    # plt.plot(dates_train, y_train, color='red', label='Training Observations')
    # plt.plot(dates_val, val_predictions, color='peachpuff', label='Validation Predictions')
    # plt.plot(dates_val, y_val, color='lightblue', label='Validation Observations')
    # plt.plot(dates_test, test_predictions, color='orange', label='Testing Predictions')
    # plt.plot(dates_test, y_test, color='blue', label='Testing Observations')

    # Create a custom legend
    # blue_line = mlines.Line2D([], [], color='blue', markersize=15, label='Testing Observations')
    # orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Predictions')
    # lightblue_line = mlines.Line2D([], [], color='lightblue', markersize=15, label='Validation Observations')
    # peachpuff_line = mlines.Line2D([], [], color='peachpuff', markersize=15, label='Validation Predictions')
    # red_line = mlines.Line2D([], [], color='red', markersize=15, label='Training Observations')
    # magenta_line = mlines.Line2D([], [], color='magenta', markersize=15, label='Training Predictions')
    # plt.legend(handles=[blue_line, orange_line, lightblue_line, peachpuff_line, red_line, magenta_line])

    # plt.title(f'{col} price')
    # plt.show()

In [ ]:
date, X, y = df_to_windowed_df(reyes_sym_data.copy(), 3)

In [ ]:
date.shape

In [ ]:
# Suppress TensorFlow logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)

# Suppress Python warnings
warnings.filterwarnings('ignore')

def LSTM_tuning(X_train, y_train, X_val, y_val):
    np.random.seed(20)
    tf.random.set_seed(20)
    # Clear TensorFlow session
    K.clear_session()
    # Function to create model
    def create_model(learn_rate=0.001, neurons=32, dropout_rate=0, activation='relu'):
        # input layers
        inputs = Input(shape=(3,1))
        x = inputs
        x = LSTM(units=neurons, activation=activation, return_sequences=True)(x)
        # middle layer(s)
        x = Dropout(dropout_rate)(x)
        x = LSTM(units=neurons, activation=activation, return_sequences=False)(x)
        x = Dense(units=neurons, activation=activation)(x)
        x = Flatten()(x)
        # last layer
        outputs = Dense(1)(x)

        model = Model(inputs=inputs, outputs=outputs)

        optimizer = Adam(learning_rate=learn_rate)
        model.compile(loss='mean_squared_error', optimizer=optimizer)

        return model

    # Wrap Keras model with KerasRegressor
    model = KerasRegressor(build_fn=create_model, learn_rate=0.001, neurons=32, dropout_rate=0.0, activation='relu', verbose=0, epochs=50, batch_size=32)

    # Define the parameter space
    search_spaces = {
        'learn_rate': (0.001, 0.01, 'log-uniform'),  # log-uniform distribution
        'neurons': [32, 64, 128],  # list of choices
        'dropout_rate': (0.0, 0.5, 'uniform'),  # uniform distribution
        'batch_size': (32, 128, 'uniform'),  # uniform distribution
    }
    # Create TimeSeriesSplit object
    tscv = TimeSeriesSplit(n_splits=3)

    # Perform Bayesian Search Optimisation
    bayes = BayesSearchCV(estimator=model, search_spaces=search_spaces, n_iter=10, cv=tscv, n_jobs=-1, random_state=20)
    bayes_result = bayes.fit(X_train, y_train)

    best_params = bayes_result.best_params_
    # Build the model using best parameters
    best_model = create_model(learn_rate=best_params['learn_rate'],
                  neurons=best_params['neurons'],
                  dropout_rate=best_params['dropout_rate'])

    # Extract the best batch size from the best parameters
    best_batch_size = best_params['batch_size']

    # Define a ModelCheckpoint callback
    cp = ModelCheckpoint('best_model_u', monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    # Fit the best model
    best_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=best_params['batch_size'], callbacks=[cp], verbose=0)

    # Load the best model
    best_model.load_weights('best_model_u')

    print("Best Hyperparameters:")
    for param, value in best_params.items():
        print(f"{param}: {value}")

    return best_model

In [ ]:
def iterative_forecasting(model, X, steps=5):
    future_values = []
    print(X.shape)
    for i in range(0, len(X) - steps + 1, steps):
        input_sequence = X[i:i+1, :].copy()
        for _ in range(steps):
            next_value = model.predict(input_sequence, verbose=0)
            future_values.append(next_value.flatten())
            input_sequence = np.roll(input_sequence, shift=-1, axis=1)
            input_sequence[0, -1] = next_value

    return np.array(future_values).reshape(-1,1)

##### step-by-step walkthrough

In [ ]:
data = reyes_sym_data.copy()
data

In [ ]:
def df_to_windowed_df(df, window_size):
    df = df.copy()
    windowed_data = []
    target_data = []
    date_data = df.index
    for i in range(window_size, len(df)):
        window = df.iloc[i-window_size:i]
        target = df.iloc[i]
        windowed_data.append(window)
        target_data.append(target)

    date_data, windowed_data, target_data = np.array(date_data), np.array(windowed_data), np.array(target_data)

    return date_data[window_size:], windowed_data, target_data

In [ ]:
date, X, y = df_to_windowed_df(data, window_size=3)
print(X.shape)
print(y.shape)
print(date.shape)

In [ ]:
y

In [ ]:
dates_train, dates_val, dates_test = traintestsplit(date, val_size=0.1, test_size=0.1)
X_train, X_val, X_test = traintestsplit(X, val_size=0.1, test_size=0.1)
y_train, y_val, y_test = traintestsplit(y, val_size=0.1, test_size=0.1)

plt.plot(dates_train, y_train)
plt.plot(dates_val, y_val)
plt.plot(dates_test, y_test)

plt.legend(['Train', 'Validation', 'Test'])
plt.ylabel('Close Price')
plt.xlabel('Time')
plt.title('Stock: SPY (S&P 500), Range: 1yr, Frequency: 1day')

In [ ]:
print(dates_train.shape)
print(dates_val.shape)
print(dates_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
model = Sequential([layers.Input((3, 1)),
                    layers.LSTM(64),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(1)])

model.summary()

model.compile(loss='mse',
              optimizer=Adam(learning_rate=0.001),
              metrics=['mean_absolute_error'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50)

In [ ]:
train_predictions = model.predict(X_train).flatten()

plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.xticks(rotation=45)
plt.legend(['Training Predictions', 'Training Observations'])

In [ ]:
val_predictions = model.predict(X_val).flatten()

plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.xticks(rotation=45)
plt.legend(['Validation Predictions', 'Validation Observations'])

In [ ]:
test_predictions = model.predict(X_test).flatten()

plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.xticks(rotation=45)
plt.legend(['Testing Predictions', 'Testing Observations'])

In [ ]:
plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Training Predictions',
            'Training Observations',
            'Validation Predictions',
            'Validation Observations',
            'Testing Predictions',
            'Testing Observations'])

##### LSTM (univariate) pipeline

In [ ]:
def LSTM_pipeline(df, window_size):
    train_predictions_df, val_predictions_df, test_predictions_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    date, X, y = df_to_windowed_df(df, window_size)
    dates_train, dates_val, dates_test = traintestsplit(date, val_size=0.1, test_size=0.1)
    X_train, X_val, X_test = traintestsplit(X, val_size=0.1, test_size=0.1)
    y_train, y_val, y_test = traintestsplit(y, val_size=0.1, test_size=0.1)

    for i, col in enumerate(df.columns):
        print('progress!')
        model = LSTM_tuning(X_train[:,:,i], y_train[:,i], X_val[:,:,i], y_val[:,i])

        train_predictions = iterative_forecasting(model, X_train[:,:,i])
        train_predictions_t = pd.DataFrame(train_predictions)
        train_predictions_df = pd.concat([train_predictions_df, train_predictions_t], axis=1)

        val_predictions = iterative_forecasting(model, X_val[:,:,i])
        val_predictions_t = pd.DataFrame(val_predictions)
        val_predictions_df = pd.concat([val_predictions_df, val_predictions_t], axis=1)

        test_predictions = iterative_forecasting(model, X_test[:,:,i])
        test_predictions_t = pd.DataFrame(test_predictions)
        test_predictions_df = pd.concat([test_predictions_df, test_predictions_t], axis=1)

    if len(df.columns) == 1:
        y_test_df, y_val_df, y_train_df = pd.DataFrame(y_test), pd.DataFrame(y_val), pd.DataFrame(y_train)
        y_test_df.columns, y_val_df.columns, y_train_df.columns, train_predictions_df.columns, val_predictions_df.columns, test_predictions_df.columns, = df.columns,df.columns,df.columns,df.columns,df.columns,df.columns
        performance(y_test_df, test_predictions_df)
        plot_LSTM(dates_train, dates_val, dates_test, train_predictions_df, val_predictions_df, test_predictions_df, y_train_df, y_val_df, y_test_df, df.columns.copy().tolist()[0])
        # test_predictions_df.to_csv(base_path + '/save_forecasts/reyes_sp_lstm_forecasted.csv', index=False)
    else:
        y_train_df = y_train
        y_val_df = y_val
        y_test_ = math_coherence(y_test)
        full_train_predictions = train_predictions_df
        full_val_predictions = val_predictions_df
        print(test_predictions_df)
        print(test_predictions_df.shape)
        print(y_test_.shape)
        full_test_predictions = math_coherence(test_predictions_df)
        print(full_test_predictions)

        truncation = len(y_test_) // 5 * 5
        y_test_df = y_test_.iloc[:truncation,:]
        dates_test_df = dates_test[:truncation]
        print(y_test_df.shape)
        print(dates_test_df.shape)

        for true, forecast in zip([y_test_df],[full_test_predictions]):
            true['min'] = (true['Q1'] - (1.5*(true['Q3'] - true['Q1'])))
            true['max'] = (true['Q3'] + (1.5*(true['Q3'] - true['Q1'])))
            forecast['min'] = (forecast['Q1'] - (1.5*(forecast['Q3'] - forecast['Q1'])))
            forecast['max'] = (forecast['Q3'] + (1.5*(forecast['Q3'] - forecast['Q1'])))

        performance(y_test_df, full_test_predictions)

        for col in y_test_df.columns:
            plot_LSTM(dates_test_df, full_test_predictions[col], y_test_df[col], col)

        # plt.plot(dates_train, full_train_predictions, color='magenta', label = 'Training Predictions')
        # plt.plot(dates_train, y_train_df, color='red', label='Training Observations')
        # plt.plot(dates_val, full_val_predictions, color='peachpuff', label='Validation Predictions')
        # plt.plot(dates_val, y_val_df, color='lightblue', label='Validation Observations')
        plt.plot(dates_test_df, y_test_df, color='orange', label='Testing Observations')
        plt.plot(dates_test_df, full_test_predictions, color='green', label='Testing Predictions')

        # Create a custom legend
        orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Observations')
        green_line = mlines.Line2D([], [], color='green', markersize=15, label='Testing Predictions')
        # lightblue_line = mlines.Line2D([], [], color='lightblue', markersize=15, label='Validation Observations')
        # peachpuff_line = mlines.Line2D([], [], color='peachpuff', markersize=15, label='Validation Predictions')
        # red_line = mlines.Line2D([], [], color='red', markersize=15, label='Training Observations')
        # magenta_line = mlines.Line2D([], [], color='magenta', markersize=15, label='Training Predictions')
        plt.legend(handles=[orange_line, green_line])
        plt.xticks(rotation=45)
        plt.title('All symbolic rates')
        # plt.grid(True)
        plt.show()

        # full_test_predictions.to_csv(base_path + '/save_forecasts/reyes_lstm_forecasted.csv', index=False)

In [ ]:
LSTM_pipeline(reyes_sym_data.copy(),window_size=3)

# Multivariate Forecasting

- VAR/VECM
- Check stationarity of individual series in multivariate series
- if all stationary, proceed with VAR.
- if some/all non-stationary, check for cointegration
- if no cointegration, convert to stationary as before and proceed with VAR.
- convert data to stationary using logarithm and differencing.
- revert predictions to the non-stationary scale using cumulative summation and exponential.
- if cointegration, proceed with VECM.

### VAR/VECM

In [ ]:
# temp = np.log(train.head(1))
# new_df = np.log(train).diff().dropna()
# new_df = pd.concat([temp, new_df])
# np.exp(new_df.cumsum())

In [ ]:
data = reyes_sym_data.copy()

In [ ]:
train, test = traintestsplit(data, val_size=0, test_size=0.1)

plt.plot(train.index, train)
plt.plot(test.index, test)
plt.legend(['Train', 'Test'])
plt.ylabel('Close Price')
plt.xlabel('Time')
plt.title('Stock: SPY (S&P 500), Range: 1yr, Frequency: 1day')

In [ ]:
train

In [ ]:
# print("--CHECK STATIONARITY--")
# check_stationarity(train.close)
# check_stationarity(train['min'])
# check_stationarity(train['q25'])
# check_stationarity(train['q50'])
# check_stationarity(train['q75'])
# check_stationarity(train['max'])

In [ ]:
# Y = train['q25']
# X1 = train['q50']
# X2 = train['q75']

# # Combine the X series into a 2D array
# X = np.column_stack((X1, X2))

# # Add a constant to X
# X = sm.add_constant(X)

# # Estimate the model
# model = sm.OLS(Y, X)
# results = model.fit()

# # Print the coefficients
# print(results.params)
# # Engle-Granger test: apply results.resid to ADF test
# check_stationarity(results.resid)

# Johansen test
# Apply the Johansen cointegration test
johansen_test = vecm.coint_johansen(train, det_order=0, k_ar_diff=1)

# Print the eigenvalues
print('Eigenvalues:\n', johansen_test.lr1)

# Print the trace statistics
print('Trace Statistics:\n', johansen_test.lr2)

# Print the critical values
print('Critical Values (90%, 95%, 99%):\n', johansen_test.cvt)


### VAR

In [ ]:
# temp = np.log(data).tail(1)
# stationary_data = np.log(data).diff().dropna()
# train, test = traintestsplit(stationary_data, val_size=0, test_size=0.1)

# stationary_data, test_stationary_data = train[1:].copy(), test.copy()
# temp = np.log(train).tail(1)
# stationary_data['q25'] = np.log(train['q25']).diff().dropna()
# temp_test = pd.concat([train['q25'].tail(1), test['q25']])
# test_stationary_data['q25'] = np.log(temp_test).diff().dropna()
# test_stationary_data

stationary_data, test_stationary_data = train.copy(), test.copy()
stationary_data['Q1'] = np.log(train['Q1'])
test_stationary_data['Q1'] = np.log(test['Q1'])

In [ ]:
# stationary_data, test_stationary_data = train[1:].copy(), test.copy()
# stationary_data['Q1'] = np.log(train['Q1']).diff().dropna()
# temp_test = np.log(pd.concat([train.tail(1), test])).diff().dropna()
# test_stationary_data['Q1'] = temp_test['Q1']

In [ ]:
plt.plot(stationary_data)
plt.xticks(rotation=45)
plt.show()

In [ ]:
var = VAR(stationary_data)

order = var.select_order()
order.summary()

In [ ]:
# Fit the VAR model, returns the Estimation results
estimation_results = var.fit(7)
# Compute output summary of estimates
estimation_results.summary()

In [ ]:
# Fetch the lag order
lag_order = estimation_results.k_ar
print(lag_order)

# in-sample predictions
pred = estimation_results.fittedvalues
pred = pd.DataFrame(pred, index=train.index)
pred.columns = ['Q1', 'Q2', 'Q3']

# out-of-sample forecasts
forecasts = []
history = stationary_data.copy()

for t in range(0,len(test)-4,5):
    # Forecast one step ahead
    forecast = estimation_results.forecast(history.values[-lag_order:], 5)
    # Append the forecast to your list of forecasts
    forecasts.extend(forecast)
    # Append the actual observation from test set to your data (to be used for the next forecast)
    for i in range(5):
        history = pd.concat([history, pd.DataFrame([test_stationary_data.iloc[t+i,:]])])

forecasts = pd.DataFrame(forecasts, index=test.index[:len(forecasts)])
forecasts.columns = ['Q1', 'Q2', 'Q3']

forecasts['Q1'] = np.exp(forecasts['Q1'])
# temp_forecast = (pd.concat([np.log(train['Q1'].tail(1)), forecasts['Q1']])).cumsum()
# forecasts['Q1'] = np.exp(temp_forecast[1:])

# pred_df = math_coherence(pred)
forecast_df = math_coherence(forecasts)
train_df = math_coherence(train)
test_df = math_coherence(test)

test_df['min'] = (test_df['Q1'] - (1.5*(test_df['Q3'] - test_df['Q1'])))
test_df['max'] = (test_df['Q3'] + (1.5*(test_df['Q3'] - test_df['Q1'])))
forecast_df['min'] = (forecast_df['Q1'] - (1.5*(forecast_df['Q3'] - forecast_df['Q1'])))
forecast_df['max'] = (forecast_df['Q3'] + (1.5*(forecast_df['Q3'] - forecast_df['Q1'])))

In [ ]:
test_df

In [ ]:
performance(test_df[:-4], forecast_df)
# forecast_df.to_csv(base_path + '/save_forecasts/eurusd_var_forecasted.csv', index=False)

In [ ]:
new_test = test_df[:-4]

In [ ]:
for col in test_df.columns:
    plt.plot(new_test.index, new_test[col], color='orange', label='Testing Observations')
    plt.plot(forecast_df.index, forecast_df[col], color='green', label='Testing Predictions')
    plt.xticks(rotation=45)
    orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Observations')
    green_line = mlines.Line2D([], [], color='green', markersize=15, label='Testing Predictions')
    plt.legend(handles=[orange_line, green_line])
    plt.title(f'{col} rates')
    plt.show()

# plt.plot(train_df.index, train_df, color='lightblue', label='train')
plt.plot(test_df.index, test_df, color='orange', label='Testing Observations')
# plt.plot(pred_df.index, pred_df, color='peachpuff', label='in-sample prediction', lw=0.9)
plt.plot(forecast_df.index, forecast_df, color='green', label='Testing Predictions', lw=0.9)
plt.xticks(rotation=45)

# Create a custom legend
# lightblue_line = mlines.Line2D([], [], color='lightblue', markersize=15, label='train')
orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Observations')
# peachpuff_line = mlines.Line2D([], [], color='peachpuff', markersize=15, label='in-sample prediction')
green_line = mlines.Line2D([], [], color='green', markersize=15, label='Testing Predictions')
plt.legend(handles=[orange_line, green_line])
plt.title('All symbolic rates')
plt.show()

### VECM

In [ ]:
model = var2(train)
results = model.select_order()
print(results.summary())

In [ ]:
lag_order = 24
# coint_rank determined from johansen test: no. of cointegrated time series -1
model = vecm.VECM(train, k_ar_diff = lag_order, coint_rank = 2, deterministic="ci")
result = model.fit()
# result.summary()

# produce in-sample forecasts
pred = result.fittedvalues
pred = pd.DataFrame(pred, index=train.index[lag_order:-1])
pred.columns = ['Q1', 'Q2', 'Q3']
# out-of-sample forecasts
forecasts = []
history = train.copy()
history = history.values.tolist()

for t in range(len(test)):
    # Forecast one step ahead
    model = vecm.VECM(history, k_ar_diff = lag_order, coint_rank = 2, deterministic="ci")
    result = model.fit()
    # Forecast one step ahead
    forecast = result.predict(steps=1)
    # Append the forecast to your list of forecasts
    forecasts.append(forecast[0])
    # Append the actual observation from test set to your data (to be used for the next forecast)
    history.append(test.iloc[t,:])
    # history = pd.concat([history, pd.DataFrame([test.iloc[t,:]])])

# forecasts = result.predict(steps=len(test))

forecasts = pd.DataFrame(forecasts, index=test.index)
forecasts.columns = ['Q1', 'Q2', 'Q3']

pred_df = math_coherence(pred)
forecast_df = math_coherence(forecasts)
train_df = math_coherence(train)
test_df = math_coherence(test)

test_df['min'] = (test_df['Q1'] - (1.5*(test_df['Q3'] - test_df['Q1'])))
test_df['max'] = (test_df['Q3'] + (1.5*(test_df['Q3'] - test_df['Q1'])))
forecast_df['min'] = (forecast_df['Q1'] - (1.5*(forecast_df['Q3'] - forecast_df['Q1'])))
forecast_df['max'] = (forecast_df['Q3'] + (1.5*(forecast_df['Q3'] - forecast_df['Q1'])))

In [ ]:
training = train_df[lag_order:-1]
predictions = pred_df.copy()
training['min'] = (training['Q1'] - (1.5*(training['Q3'] - training['Q1'])))
training['max'] = (training['Q3'] + (1.5*(training['Q3'] - training['Q1'])))
predictions['min'] = (predictions['Q1'] - (1.5*(predictions['Q3'] - predictions['Q1'])))
predictions['max'] = (predictions['Q3'] + (1.5*(predictions['Q3'] - predictions['Q1'])))

performance(training, predictions)

In [ ]:
performance(test_df, forecast_df)
# forecast_df.to_csv(base_path + '/save_forecasts/btc_vecm_forecasted.csv', index=False)

In [ ]:
for col in test_df.columns:
    plt.plot(test_df.index, test_df[col], color='orange', label='Testing Observations')
    plt.plot(forecast_df.index, forecast_df[col], color='green', label='Testing Predictions')
    plt.xticks(rotation=45)
    orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Observations')
    green_line = mlines.Line2D([], [], color='green', markersize=15, label='Testing Predictions')
    plt.legend(handles=[orange_line, green_line])
    plt.title(f'{col} prices')
    plt.show()

# plt.plot(train_df.index, train_df, color='lightblue', label='train')
plt.plot(test_df.index, test_df, color='orange', label='Testing Observations')
# plt.plot(pred_df.index, pred_df, color='peachpuff', label='in-sample prediction', lw=0.9)
plt.plot(forecast_df.index, forecast_df, color='green', label='Testing Predictions', lw=0.9)
plt.xticks(rotation=45)

# Create a custom legend
# lightblue_line = mlines.Line2D([], [], color='lightblue', markersize=15, label='train')
orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Observations')
# peachpuff_line = mlines.Line2D([], [], color='peachpuff', markersize=15, label='in-sample prediction')
green_line = mlines.Line2D([], [], color='green', markersize=15, label='Testing Predictions')
plt.legend(handles=[orange_line, green_line])
plt.title('All symbolic prices')
plt.show()

### LSTM

In [ ]:
data = reyes_sym_data.copy()

In [ ]:
def df_to_windowed_df(df, window_size):
    df = df.copy()
    windowed_data = []
    target_data = []
    date_data = df.index
    for i in range(window_size, len(df)):
        window = df.iloc[i-window_size:i]
        target = df.iloc[i]
        windowed_data.append(window)
        target_data.append(target)

    windowed_data, target_data = np.array(windowed_data), np.array(target_data)

    return date_data[window_size:], windowed_data, target_data

In [ ]:
date, X, y = df_to_windowed_df(data, window_size=3)
X = np.reshape(X,(X.shape[0], X.shape[1], X.shape[2]))
print(X.shape)
print(y.shape)
print(date.shape)

In [ ]:
dates_train, dates_val, dates_test = traintestsplit(date, val_size=0.1, test_size=0.1)
X_train, X_val, X_test = traintestsplit(X, val_size=0.1, test_size=0.1)
y_train, y_val, y_test = traintestsplit(y, val_size=0.1, test_size=0.1)

# 0 = min variable... 4 = max variable
for i in range(3):
  plt.plot(dates_train, y_train[:,i])
  plt.plot(dates_val, y_val[:,i])
  plt.plot(dates_test, y_test[:,i])
  plt.xticks(rotation=45)

plt.legend(['Train', 'Validation', 'Test'])
plt.ylabel('Close Price')
plt.xlabel('Time')
plt.title('Stock: SPY (S&P 500), Range: 1yr, Frequency: 1day')

In [ ]:
print(dates_train.shape)
print(dates_val.shape)
print(dates_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
# Suppress TensorFlow logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)

# Suppress Python warnings
warnings.filterwarnings('ignore')

def LSTM_tuning(X_train, y_train, X_val, y_val):
    np.random.seed(20)
    tf.random.set_seed(20)
    # Clear TensorFlow session
    K.clear_session()
    # Function to create model
    def create_model(learn_rate=0.001, neurons=32, dropout_rate=0, activation='relu'):
        # input layers
        inputs = Input(shape=(3,3))
        x = inputs
        x = LSTM(units=neurons, activation=activation, return_sequences=True)(x)
        # middle layer(s)
        x = Dropout(dropout_rate)(x)
        x = LSTM(units=neurons, activation=activation, return_sequences=False)(x)
        x = Dense(units=neurons, activation=activation)(x)
        # last layer
        outputs = Dense(3)(x)

        model = Model(inputs=inputs, outputs=outputs)

        optimizer = Adam(learning_rate=learn_rate)
        model.compile(loss='mean_squared_error', optimizer=optimizer)

        return model

    # Wrap Keras model with KerasRegressor
    model = KerasRegressor(build_fn=create_model, learn_rate=0.001, neurons=32, dropout_rate=0.0, activation='relu', verbose=0, epochs=50, batch_size=32)

    # Define the parameter space
    search_spaces = {
        'learn_rate': (0.001, 0.01, 'log-uniform'),  # log-uniform distribution
        'neurons': [32, 64, 128],  # list of choices
        'dropout_rate': (0.0, 0.5, 'uniform'),  # uniform distribution
        'batch_size': (32, 128, 'uniform'),  # uniform distribution
    }
    # Create TimeSeriesSplit object
    tscv = TimeSeriesSplit(n_splits=3)

    # Perform Bayesian Search Optimisation
    bayes = BayesSearchCV(estimator=model, search_spaces=search_spaces, n_iter=10, cv=tscv, n_jobs=-1, random_state=20)
    bayes_result = bayes.fit(X_train, y_train)

    best_params = bayes_result.best_params_
    # Build the model using best parameters
    best_model = create_model(learn_rate=best_params['learn_rate'],
                  neurons=best_params['neurons'],
                  dropout_rate=best_params['dropout_rate'])

    # Extract the best batch size from the best parameters
    best_batch_size = best_params['batch_size']

    # Define a ModelCheckpoint callback
    cp = ModelCheckpoint('best_model_m', monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    # Fit the best model
    best_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=best_params['batch_size'], callbacks=[cp], verbose=0)

    # Load the best model
    best_model.load_weights('best_model_m')

    print("Best Hyperparameters:")
    for param, value in best_params.items():
        print(f"{param}: {value}")

    return best_model

def iterative_forecasting(model, X, steps=5):
    future_values = []
    print(X.shape)
    for i in range(0, len(X) - steps + 1, steps):
        input_sequence = X[i:i+1, :, :].copy()  # Take one sample from X
        for _ in range(steps):
            next_value = model.predict(input_sequence, verbose=0)
            future_values.append(next_value.flatten())
            input_sequence = np.roll(input_sequence, shift=-1, axis=1)
            input_sequence[0, -1, :] = next_value  # Assuming a univariate time series

    return np.array(future_values).reshape(-1,3)

In [ ]:
model = LSTM_tuning(X_train, y_train, X_val, y_val)
# train_predictions = model.predict(X_train)
# train_predictions_df = math_coherence(train_predictions)
# y_train_df = math_coherence(y_train)
# plt.plot(dates_train, train_predictions_df, color='orange', label='Training Predictions')
# plt.plot(dates_train, y_train_df, color='blue', label='Training Observations')
# plt.xticks(rotation=45)
# # Create a custom legend
# blue_line = mlines.Line2D([], [], color='blue', markersize=15, label='Training Observations')
# orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Training Predictions')
# plt.legend(handles=[blue_line, orange_line])

In [ ]:
# val_predictions = model.predict(X_val)

# val_predictions_df = math_coherence(val_predictions)
# y_val_df = math_coherence(y_val)

# plt.plot(dates_val, val_predictions_df, color='orange', label='Validation Predictions')
# plt.plot(dates_val, y_val_df, color='blue', label='Validation Observations')
# plt.xticks(rotation=45)
# # Create a custom legend
# blue_line = mlines.Line2D([], [], color='blue', markersize=15, label='Validation Observations')
# orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Validation Predictions')
# plt.legend(handles=[blue_line, orange_line])

In [ ]:
test_predictions = model.predict(X_test)
test_predictions_df = math_coherence(test_predictions)

y_test_df = math_coherence(y_test)

y_test_df['min'] = (y_test_df['Q1'] - (1.5*(y_test_df['Q3'] - y_test_df['Q1'])))
y_test_df['max'] = (y_test_df['Q3'] + (1.5*(y_test_df['Q3'] - y_test_df['Q1'])))
test_predictions_df['min'] = (test_predictions_df['Q1'] - (1.5*(test_predictions_df['Q3'] - test_predictions_df['Q1'])))
test_predictions_df['max'] = (test_predictions_df['Q3'] + (1.5*(test_predictions_df['Q3'] - test_predictions_df['Q1'])))

performance(y_test_df, test_predictions_df)

# test_predictions_df.to_csv(base_path + '/save_forecasts/eurusd_lstm_m_forecasted.csv', index=False)

for col in y_test_df.columns:
    plt.plot(dates_test, y_test_df[col], color='orange', label='Testing Observations')
    plt.plot(dates_test, test_predictions_df[col], color='green', label='Testing Predictions')
    plt.xticks(rotation=45)
    orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Observations')
    green_line = mlines.Line2D([], [], color='green', markersize=15, label='Testing Predictions')
    plt.legend(handles=[orange_line, green_line])
    plt.title(f'{col} rates')
    plt.show()

plt.plot(dates_test, y_test_df, color='orange', label='Testing Observations')
plt.plot(dates_test, test_predictions_df, color='green', label='Testing Predictions')
plt.xticks(rotation=45)
# Create a custom legend
orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Observations')
green_line = mlines.Line2D([], [], color='green', markersize=15, label='Testing Predictions')
plt.title('All symbolic rates')
plt.legend(handles=[orange_line, green_line])

In [ ]:
# plt.plot(dates_train, train_predictions_df, color='magenta', label = 'Training Predictions')
# plt.plot(dates_train, y_train_df, color='red', label='Training Observations')
# plt.plot(dates_val, val_predictions_df, color='peachpuff', label='Validation Predictions')
# plt.plot(dates_val, y_val_df, color='lightblue', label='Validation Observations')
# plt.plot(dates_test, test_predictions_df, color='orange', label='Testing Predictions')
# plt.plot(dates_test, y_test_df, color='blue', label='Testing Observations')

# # Create a custom legend
# blue_line = mlines.Line2D([], [], color='blue', markersize=15, label='Testing Observations')
# orange_line = mlines.Line2D([], [], color='orange', markersize=15, label='Testing Predictions')
# lightblue_line = mlines.Line2D([], [], color='lightblue', markersize=15, label='Validation Observations')
# peachpuff_line = mlines.Line2D([], [], color='peachpuff', markersize=15, label='Validation Predictions')
# red_line = mlines.Line2D([], [], color='red', markersize=15, label='Training Observations')
# magenta_line = mlines.Line2D([], [], color='magenta', markersize=15, label='Training Predictions')
# plt.legend(handles=[blue_line, orange_line, lightblue_line, peachpuff_line, red_line, magenta_line])

# Buy/Sell Trading Simulation

In [ ]:
def sym_trading_simulation(observed, forecasted):
    initial_capital = 10000
    # Lists to store timestamps of buy, sell, and hold actions
    buy_dates = []
    sell_dates = []
    hold_dates = []

    # Initialize variables
    capital = initial_capital
    asset_held = 0  # Number of units of the asset held
    num_buys = 0
    num_sells = 0

    # Iterate through the data values
    for date in range(1, len(forecasted)):  # Start from 1 because we need previous timestep's data
        # Buy conditions
        # Strong conditions
        if forecasted['min'].iloc[date] > observed['max'].iloc[date-1]:
            # Strong Buy
            asset_held += capital / observed['Q2'].iloc[date]
            capital = 0
            num_buys += 1
            buy_dates.append(forecasted.index[date])
        elif forecasted['max'].iloc[date] < observed['min'].iloc[date-1]:
            # Strong Sell
            capital += asset_held * observed['Q2'].iloc[date]
            asset_held = 0
            num_sells += 1
            sell_dates.append(forecasted.index[date])
        # Moderate-Strong conditions
        elif forecasted['min'].iloc[date] > observed['Q3'].iloc[date-1] or forecasted['Q1'].iloc[date] > observed['max'].iloc[date-1]:
            # Moderate-Strong Buy
            asset_held += (0.75 * capital) / observed['Q2'].iloc[date]
            capital *= 0.25
            num_buys += 1
            buy_dates.append(forecasted.index[date])
        elif forecasted['max'].iloc[date] < observed['Q1'].iloc[date-1] or forecasted['Q3'].iloc[date] < observed['min'].iloc[date-1]:
            # Moderate-Strong Sell
            capital += (0.75 * asset_held) * observed['Q2'].iloc[date]
            asset_held *= 0.25
            num_sells += 1
            sell_dates.append(forecasted.index[date])
        # Weak-Moderate conditions
        elif (forecasted['min'].iloc[date] > observed['Q2'].iloc[date-1] or
              forecasted['Q1'].iloc[date] > observed['Q3'].iloc[date-1] or
              forecasted['Q2'].iloc[date] > observed['max'].iloc[date-1]):
            # Weak-Moderate Buy
            asset_held += (0.50 * capital) / observed['Q2'].iloc[date]
            capital *= 0.50
            num_buys += 1
            buy_dates.append(forecasted.index[date])
        elif (forecasted['max'].iloc[date] < observed['Q2'].iloc[date-1] or
              forecasted['Q3'].iloc[date] < observed['Q1'].iloc[date-1] or
              forecasted['Q2'].iloc[date] < observed['min'].iloc[date-1]):
            # Weak-Moderate Sell
            capital += (0.50 * asset_held) * observed['Q2'].iloc[date]
            asset_held *= 0.50
            num_sells += 1
            sell_dates.append(forecasted.index[date])
        # Weak conditions
        elif (forecasted['min'].iloc[date] > observed['Q1'].iloc[date-1] or
              forecasted['Q1'].iloc[date] > observed['Q2'].iloc[date-1] or
              forecasted['Q2'].iloc[date] > observed['Q3'].iloc[date-1] or
              forecasted['Q3'].iloc[date] > observed['max'].iloc[date-1]):
            # Weak Buy
            asset_held += (0.25 * capital) / observed['Q2'].iloc[date]
            capital *= 0.75
            num_buys += 1
            buy_dates.append(forecasted.index[date])
        elif (forecasted['max'].iloc[date] < observed['Q3'].iloc[date-1] or
              forecasted['Q3'].iloc[date] < observed['Q2'].iloc[date-1] or
              forecasted['Q2'].iloc[date] < observed['Q1'].iloc[date-1] or
              forecasted['Q1'].iloc[date] < observed['min'].iloc[date-1]):
            # Weak Sell
            capital += (0.25 * asset_held) * observed['Q2'].iloc[date]
            asset_held *= 0.75
            num_sells += 1
            sell_dates.append(forecasted.index[date])
        else:
            # Hold
            hold_dates.append(forecasted.index[date])
    # If there's any asset left at the end, sell it
    capital += asset_held * observed['Q2'].iloc[-1]

    # Calculate final capital and profit or loss
    final_capital = capital
    profit_or_loss = final_capital - initial_capital

    # Plotting
    plt.figure(figsize=(10, 6))
    observed['Q2'].plot(label='Observed Median (Q2)', color='blue')
    forecasted['Q2'].plot(label='Forecasted Median (Q2)', color='orange')
    plt.fill_between(observed.index, observed['min'], observed['max'], color='cyan', alpha=0.3, label='Observed Min-Max range')
    plt.fill_between(forecasted.index, forecasted['min'], forecasted['max'], color='yellow', alpha=0.3, label='Forecasted Min-Max range')
    plt.scatter(buy_dates, forecasted['Q2'][buy_dates], marker='^', color='g', label='Buy Signal', alpha=1)
    plt.scatter(sell_dates, forecasted['Q2'][sell_dates], marker='v', color='r', label='Sell Signal', alpha=1)
    plt.scatter(hold_dates, forecasted['Q2'][hold_dates], marker='o', color='gray', label='Hold Signal', alpha=0.5, s=50)
    plt.legend()
    plt.title('Boxplot Aggregated Time Series Trading Simulation')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.show()

    return final_capital, profit_or_loss, num_buys, num_sells, buy_dates, sell_dates, hold_dates

def sp_trading_simulation(observed, forecasted):
    observed = pd.Series(observed)
    forecasted = pd.Series(forecasted)

    # Initialize variables
    initial_capital = 10000
    capital = initial_capital
    asset_held = 0  # Number of units of the asset held
    num_buys = 0
    num_sells = 0

    # Lists to store timestamps of buy, sell, and hold actions
    buy_dates = []
    sell_dates = []
    hold_dates = []

    # Iterate through the observed and forecasted values
    for date in range(1, len(forecasted)):
        if forecasted.iloc[date] > 1.01 * observed.iloc[date-1]:  # Buy signal
            # Buy as much as possible
            asset_held += capital / observed.iloc[date]
            capital = 0
            num_buys += 1
            buy_dates.append(observed.index[date])
        elif forecasted.iloc[date] < 0.99 * observed.iloc[date-1]:  # Sell signal
            # Sell all assets
            capital += asset_held * observed.iloc[date]
            asset_held = 0
            num_sells += 1
            sell_dates.append(observed.index[date])
        else:  # Hold
            hold_dates.append(observed.index[date])

    # If there's any asset left at the end, sell it
    capital += asset_held * observed.iloc[-1]

    # Calculate final capital and profit or loss
    final_capital = capital
    profit_or_loss = final_capital - initial_capital

    # Plotting
    plt.figure(figsize=(10, 6))
    observed.plot(label='Observed', color='blue')
    forecasted.plot(label='Forecasted', color='orange')
    plt.scatter(buy_dates, forecasted[buy_dates], marker='^', color='g', label='Buy Signal', alpha=1)
    plt.scatter(sell_dates, forecasted[sell_dates], marker='v', color='r', label='Sell Signal', alpha=1)
    plt.scatter(hold_dates, forecasted[hold_dates], marker='o', color='gray', label='Hold Signal', alpha=0.5, s=50)
    plt.legend()
    plt.title('Mean-Aggregated Time Series Trading Simulation')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.show()

    return final_capital, profit_or_loss, num_buys, num_sells, buy_dates, sell_dates, hold_dates

def baseline(observed, forecasted):
    # Initial capital
    capital = 10000

    # Buy on the first date using all capital at Q2 price
    asset_held = capital / observed['Q2'].iloc[0]
    capital = 0  # Set capital to zero after the buy

    # Sell on the last date using all assets at Q2_end price
    Q2_end = observed['Q2'].iloc[-1]
    capital = asset_held * Q2_end
    asset_held = 0

    # Calculate profit or loss
    profit_or_loss = capital - 10000

    return capital, profit_or_loss

def baseline_sp(observed, forecasted):
    # Initial capital
    capital = 10000

    # Buy on the first date using all capital at Q2 price
    asset_held = capital / observed.iloc[0]
    capital = 0  # Set capital to zero after the buy

    # Sell on the last date using all assets at Q2_end price
    close_end = observed.iloc[-1]
    capital = asset_held * close_end
    asset_held = 0

    # Calculate profit or loss
    profit_or_loss = capital - 10000

    return capital, profit_or_loss

In [ ]:
forecast = pd.read_csv(base_path + '/save_forecasts/btc_arima_ms_forecasted.csv')
data = BTC_sym_data
train, test_df = traintestsplit(data, val_size=0, test_size=0.1)
test = math_coherence(test_df)

test['min'] = (test['Q1'] - (1.5*(test['Q3'] - test['Q1'])))
test['max'] = (test['Q3'] + (1.5*(test['Q3'] - test['Q1'])))
# forecast['min'] = (forecast['Q1'] - (1.5*(forecast['Q3'] - forecast['Q1'])))
# forecast['max'] = (forecast['Q3'] + (1.5*(forecast['Q3'] - forecast['Q1'])))
forecast.index = test.index
forecast.rename(columns={'q25':'Q1', 'q50':'Q2','q75':'Q3'}, inplace=True)
print(test)
print(forecast)

In [ ]:
forecast_sp = pd.read_csv(base_path + '/save_forecasts/btc_arima_sp_ms_forecasted.csv')
data_sp = BTC_sp_data
train_sp, test_sp = traintestsplit(data_sp, val_size=0, test_size=0.1)
forecast_sp.index = test_sp.index
print(test_sp)
print(forecast_sp)

In [ ]:
final_capital, pnl = baseline(test, forecast)
print(f"Final Capital: ${final_capital:.2f}")
print(f"Profit/Loss: ${pnl:.2f}")

final_capital_sp, pnl_sp = baseline_sp(test_sp, forecast_sp)
print(final_capital_sp)
print(pnl_sp)

In [ ]:
sp_trading_simulation(test_sp['close'], forecast_sp['close'])

In [ ]:
sym_trading_simulation(test, forecast)